<a href="https://colab.research.google.com/github/PyungkangHong094/timeseries/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from datetime import datetime
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# univariate one step problem with mlp
# from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import TimeseriesGenerator

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

In [5]:
data = pd.read_csv('nyc_taxi_3.csv')

In [7]:
data.set_index('timestamp')[['value', 'temp', 'precipitation', 'snowfall', 'windspeed']]
data_input = data[['value', 'temp', 'precipitation', 'snowfall', 'windspeed']]

In [8]:
data

,timestamp,value,temp,precipitation,snowfall,windspeed
0,2014.7.1 0:00,10844,14.370529,0.0,0.00,4.104631
1,2014.7.1 1:00,6210,13.450529,0.0,0.00,4.104631
2,2014.7.1 2:00,3820,12.840529,0.0,0.00,3.396233
3,2014.7.1 3:00,2369,12.160529,0.0,0.00,4.394360
4,2014.7.1 4:00,2221,11.540529,0.0,0.00,5.959060
...,...,...,...,...,...,...
5155,2015.1.31 19:00,28804,2.700529,0.2,0.14,20.929596
5156,2015.1.31 20:00,24985,2.030528,0.0,0.00,20.880001
5157,2015.1.31 21:00,23719,1.660529,0.0,0.00,19.469976
5158,2015.1.31 22:00,25721,1.180529,0.0,0.00,15.379206


In [9]:
data_input

,value,temp,precipitation,snowfall,windspeed
0,10844,14.370529,0.0,0.00,4.104631
1,6210,13.450529,0.0,0.00,4.104631
2,3820,12.840529,0.0,0.00,3.396233
3,2369,12.160529,0.0,0.00,4.394360
4,2221,11.540529,0.0,0.00,5.959060
...,...,...,...,...,...
5155,28804,2.700529,0.2,0.14,20.929596
5156,24985,2.030528,0.0,0.00,20.880001
5157,23719,1.660529,0.0,0.00,19.469976
5158,25721,1.180529,0.0,0.00,15.379206


### 1. load data
### 2. split into train/test by time
### 3.1. train scaler_x on the x_train (use StandardScaler!)
3.2. train scaler_y on the y_train

4.1. Apply scaler_x to x_train and x_test => x_train_scaled, x_test_scaled
4.2. Apply scaler_y to y_train and y_test => y_train_scaled, y_test_scaled
5. Make TimeseriesGenerator for x_train_scaled and y_train_scaled for the window=168 => x_train_window, y_train_window
6. Train https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html using x_train_window as inputs and y_train_window as target => model
7. Compute predictions of the model for scaled datasets => pred_train_scaled, pred_test_scaled
8. Reverse predictions using function inverse_transform for the scaler => pred_train, pred_test
9. Compute evaluations: MSE and MAPE



In [11]:
# Load sunspot data
from pandas import read_csv
from matplotlib import pyplot
df = read_csv('nyc_taxi_3.csv')
print(df.head())
# series.plot()
pyplot.show()

       timestamp  value       temp  precipitation  snowfall  windspeed
0  2014.7.1 0:00  10844  14.370529            0.0       0.0   4.104631
1  2014.7.1 1:00   6210  13.450529            0.0       0.0   4.104631
2  2014.7.1 2:00   3820  12.840529            0.0       0.0   3.396233
3  2014.7.1 3:00   2369  12.160529            0.0       0.0   4.394360
4  2014.7.1 4:00   2221  11.540529            0.0       0.0   5.959060


In [12]:
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [13]:
##################################### ###################################

In [29]:
from pandas import read_csv
series = read_csv('nyc_taxi_3.csv', header=0, index_col=0)
X = series.values
train_size = int(len(X) * 0.8)
train, test = X[0:train_size], X[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(train)))
print('Testing Observations: %d' % (len(test)))

Observations: 5160
Training Observations: 4128
Testing Observations: 1032


In [30]:
from pandas import read_csv
series = read_csv('nyc_taxi_3.csv', header=0, index_col=0)
X = series.values
train_size = int(len(X) * 0.2)
y_train, y_test = X[0:train_size], X[train_size:len(X)]
print('Observations: %d' % (len(X)))
print('Training Observations: %d' % (len(y_train)))
print('Testing Observations: %d' % (len(y_test)))

Observations: 5160
Training Observations: 1032
Testing Observations: 4128


In [15]:
train.shape

(4128, 5)

In [16]:
test.shape

(1032, 5)

In [17]:
train # 나누어준 트레인

array([[1.0844000e+04, 1.4370529e+01, 0.0000000e+00, 0.0000000e+00,
        4.1046314e+00],
       [6.2100000e+03, 1.3450529e+01, 0.0000000e+00, 0.0000000e+00,
        4.1046314e+00],
       [3.8200000e+03, 1.2840529e+01, 0.0000000e+00, 0.0000000e+00,
        3.3962330e+00],
       ...,
       [2.6280000e+04, 1.1670529e+01, 0.0000000e+00, 0.0000000e+00,
        3.5834015e+01],
       [2.6905000e+04, 1.1420529e+01, 1.0000000e-01, 0.0000000e+00,
        3.5647274e+01],
       [2.5807000e+04, 1.0860529e+01, 1.0000000e-01, 0.0000000e+00,
        3.5452232e+01]])

In [18]:
test # 나누어준 테스트

array([[2.5976000e+04, 9.7205290e+00, 0.0000000e+00, 0.0000000e+00,
        1.8089775e+01],
       [2.2993000e+04, 8.2905290e+00, 0.0000000e+00, 0.0000000e+00,
        1.5077082e+01],
       [1.9390000e+04, 7.5705285e+00, 0.0000000e+00, 0.0000000e+00,
        1.4241630e+01],
       ...,
       [2.3719000e+04, 1.6605287e+00, 0.0000000e+00, 0.0000000e+00,
        1.9469976e+01],
       [2.5721000e+04, 1.1805286e+00, 0.0000000e+00, 0.0000000e+00,
        1.5379206e+01],
       [2.6591000e+04, 5.3052866e-01, 0.0000000e+00, 0.0000000e+00,
        1.6548256e+01]])

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

train_scaler = preprocessing.StandardScaler().fit(train)
test_scaler = preprocessing.StandardScaler().fit(test)

In [36]:
y_train_scaler = preprocessing.StandardScaler().fit(y_train)
y_test_scaler = preprocessing.StandardScaler().fit(y_test)

In [20]:
print(train_scaler)
print(test_scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)


In [21]:
train_scaler.transform(train) # 스케일러로 받은 트레인값

array([[-0.65804515, -0.10507705, -0.1799925 , -0.02696799, -0.85104288],
       [-1.33182169, -0.25213587, -0.1799925 , -0.02696799, -0.85104288],
       [-1.67932405, -0.34964227, -0.1799925 , -0.02696799, -0.96832281],
       ...,
       [ 1.58632573, -0.53666273, -0.1799925 , -0.02696799,  4.40196148],
       [ 1.67719978, -0.57662437,  0.08283348, -0.02696799,  4.37104531],
       [ 1.51755225, -0.66613843,  0.08283348, -0.02696799,  4.33875484]])

In [22]:
train_scaler.transform(test) # 스케일러로 받은 테스트 값

array([[ 1.54212459, -0.8483635 , -0.1799925 , -0.02696799,  1.46428796],
       [ 1.10840094, -1.07694407, -0.1799925 , -0.02696799,  0.96551718],
       [ 0.58453023, -1.19203366, -0.1799925 , -0.02696799,  0.82720271],
       ...,
       [ 1.21396023, -2.13672674, -0.1799925 , -0.02696799,  1.69278915],
       [ 1.50504798, -2.2134531 , -0.1799925 , -0.02696799,  1.01553576],
       [ 1.63154466, -2.31735335, -0.1799925 , -0.02696799,  1.20907954]])

In [23]:

# scaler = StandardScaler()
# scaler.fit(train)

In [40]:
test_scaled = train_scaler.transform(test)
train_scaled =train_scaler.transform(train)

In [41]:
# y_test_scaled = train_scaler.transform(y_test)
# y_train_scaled =train_scaler.transform(y_train)

In [43]:
# y_train_scaled.shape

(1032, 5)

In [25]:

train.shape

(4128, 5)

In [26]:
train_scaled.shape

(4128, 5)

In [27]:
train.shape

(4128, 5)

In [28]:
test_scaled.shape

(1032, 5)

In [ ]:
len(train_scaled)

4128

In [ ]:
train_scaled.shape

(4128, 5)

In [ ]:
train_scaled[:,0]

array([-0.65804515, -1.33182169, -1.67932405, ...,  1.58632573,
        1.67719978,  1.51755225])

### Make TimeseriesGenerator for x_train_scaled and y_train_scaled for the window=168 => x_train_window, y_train_window

In [70]:
batch_size = 32
lengthwind = 338
# 168 - > ARIMA (168 , 0 , 0)
train_generator = TimeseriesGenerator(train_scaled[:,0], train_scaled[:,0], length= 168, sampling_rate=1, batch_size=32)
test_generator = TimeseriesGenerator(test_scaled[:,0], test_scaled[:,0], length=168, sampling_rate=1, batch_size=32)

In [71]:
# train_generator[1]
# test_generator[0]
train_generator

In [69]:
model = Sequential()
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
history = model.fit_generator(train_generator, epochs=100).history

Epoch 1/100
119/119 [==============================] - 0s 1ms/step - loss: 0.7529
Epoch 2/100
119/119 [==============================] - 0s 1ms/step - loss: 0.2358
Epoch 3/100
119/119 [==============================] - 0s 1ms/step - loss: 0.1276
Epoch 4/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0936
Epoch 5/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0782
Epoch 6/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0675
Epoch 7/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0630
Epoch 8/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0555
Epoch 9/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0526
Epoch 10/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0506
Epoch 11/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0470
Epoch 12/100
119/119 [==============================] - 0s 1ms/step - loss: 0.0434
Epoch 13/100


In [60]:
model.evaluate_generator(test_generator)
trainPredict = model.predict_generator(train_generator)
testPredict = model.predict_generator(test_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.
Instructions for updating:
Please use Model.predict, which supports generators.


In [64]:
trainPredict.shape[0]

3790

In [65]:
testPredict.shape[0]

694

In [67]:
df_final = data_input[testPredict.shape[0]*-1:]  #80%

In [68]:
df_final.count()

value            694
temp             694
precipitation    694
snowfall         694
windspeed        694
dtype: int64

In [ ]:
train_scaled

array([[-0.65804515, -0.10507705, -0.1799925 , -0.02696799, -0.85104288],
       [-1.33182169, -0.25213587, -0.1799925 , -0.02696799, -0.85104288],
       [-1.67932405, -0.34964227, -0.1799925 , -0.02696799, -0.96832281],
       ...,
       [ 1.58632573, -0.53666273, -0.1799925 , -0.02696799,  4.40196148],
       [ 1.67719978, -0.57662437,  0.08283348, -0.02696799,  4.37104531],
       [ 1.51755225, -0.66613843,  0.08283348, -0.02696799,  4.33875484]])

In [ ]:
# for i in range(len(test_generator)):
# 	x, y = test_generator[i]
# 	print('%s => %s' % (x, y))

In [50]:
model = tf.keras.Sequential()
model.add(Dense(168, activation='relu', input_dim=lengthwind))
model.add(Dense(336, activation='relu', input_dim=lengthwind,))
model.add(Dense(64))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')


NameError: ignored

In [ ]:
# fit model
model.fit_generator(test_generator, steps_per_epoch=200, verbose=0)

In [ ]:
# reshape to [10, 1]
# n_features = 1
# series = series.reshape((len(test_generator), n_features))

In [44]:
lm = LinearRegression(fit_intercept=True, normalize=True, n_jobs=None)

In [45]:
lm.fit(train_scaled,train_scaled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [47]:
accuracy = lm.score(train_scaled, train_scaled)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning:

The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').



In [48]:
print( "Linear Regression test file accuracy:"+str(accuracy))

Linear Regression test file accuracy:1.0


In [49]:
lm.coef_

array([[ 1.00000000e+00, -1.18819220e-16,  1.83546301e-16,
        -1.83907472e-16, -4.43777465e-17],
       [ 1.92424996e-16,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -2.21182293e-16],
       [ 2.46256030e-16, -6.91194664e-17,  1.00000000e+00,
        -8.20793664e-17, -1.90078533e-17],
       [-8.56434216e-17,  2.76477866e-17,  1.58110779e-16,
         1.00000000e+00, -1.24415040e-16],
       [ 1.61261424e-17, -1.10591146e-16, -3.55965252e-16,
        -3.45597332e-17,  1.00000000e+00]])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 168)               56952     
_________________________________________________________________
dense_9 (Dense)              (None, 336)               56784     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                21568     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 65        
Total params: 135,369
Trainable params: 135,369
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit_generator(generator=test_generator,steps_per_epoch=len(test_generator))
print('samples: %d' % len(test_generator))
print('samples: %d' % len(train_generator))


22/22 [==============================] - 0s 3ms/step - loss: 0.6726
samples: 22
samples: 119


In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, mode='min')

model.compile(loss=tf.losses.MeanSquaredError(),  optimizer=tf.optimizers.Adam(),   metrics=[tf.metrics.MeanAbsolutePercentageError()])

history = model.fit_generator(train_generator, epochs=200, validation_data=test_generator,  shuffle=False, callbacks=[early_stopping])

ValueError: ignored

In [ ]:
model.summary()

In [ ]:
# estimate accuracy on test data set using loaded weights
model.evaluate_generator(test_generator, verbose=0)

In [ ]:
y_test_pred = model.predict_generator(test_generator)
x_test_pred = model.predict_generator(generator=train_generator)

In [ ]:
y_test_pred.shape[0]

In [ ]:
y_test_pred.shape

In [ ]:
y_test_pred

In [ ]:
# x_test_pred

In [ ]:
x_test_pred

In [ ]:
len(x_test_pred) #3790 test-> 1032

In [ ]:
len(y_test_pred) #694

In [ ]:
df_pred = pd.concat([pd.DataFrame(y_test_pred), pd.DataFrame(train[:,1:][lengthwind:])],axis=1)

In [ ]:
rev_trans=scaler.inverse_transform(df_pred)

In [ ]:
df_final = data_input[y_test_pred.shape[0]*-1:]

In [ ]:
df_final.count()

In [ ]:
# df_final.apply(lambda col: col.drop_duplicates().reset_index(drop=True))

In [ ]:
df_final['Passenger_Pred_value'] = rev_trans[:,0]

In [ ]:
################################################################################################################

In [ ]:
#example 

In [ ]:
def mean_square_error(y_true, y_pred):
    y_true = np.asmatrix(y_true).reshape(-1)
    y_pred = np.asmatrix(y_pred).reshape(-1)

    return np.square(np.subtract(y_true, y_pred)).mean()

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    if len(np.where(y_true == 0)[0]) > 0:
        return np.inf
    else:
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100